## Enabling interactivity with Slash Commands

By enabling Slash Commands, your app can be summoned by users from any conversation in Slack.

## What are Slash Commands?

**Slash Commands** allow users to invoke your app by typing a string into the message composer box.

A submitted Slash Command will cause a payload of data to be sent from Slack to the associated app. The app can then respond in whatever way it wants using the context provided by that payload.

These commands are the entry points for complex workflows, integrations with external services, or even just simple message responses. They're the knock at your app's front door that could be the start of a great conversation.

Some commands like `/topic` and `/remind` are built into Slack ([here's a list](https://slack.com/intl/en-gb/help/articles/201259356-Slash-commands-in-Slack) of all of them) but others are created by developers like you for use with Slack Apps.

Slash Commands require a particular invocation structure that makes them less universally usable compared to other app entry points. Make sure you understand your app's audience before implementation.

## Understanding the structure of Slash Commands 

Let's look at an example Slash Command, for a simple app that stores a list of to-do tasks:

`/addtodo Blockchain`

Here's the structure:

* `/addtodo` - this is the command, the part that tells Slack to treat it as a Slash Command and where to route it. You'll define yours below.

* `Blockchain` - this is the `text` portion, it includes everything after the first space following the command. It is treated as a single parameter that is passed to the app that owns the command.

We want to make sure that Blockchain, so read on to find out how to setup commands for your apps, and how to handle and respond to them.

## Getting Started with Slash Commands 

In order to get Slash Commands up and running with your app, you'll have to create the command itself, then prepare your app to be able to handle the interaction flow. We'll describe that flow in more detail in the steps below, but the basic pattern is:

* A user in Slack types in the message box with the command, and submits it.
* A payload is sent via an HTTP POST request to your app.
* Your app responds in some way.

That's all pretty orderly, so let's look at the recipe for making a great Slash Command.

### 1. Creating a App

Creating a command is really simple, you just need two things - a Slack App and the name of your new command. If you don't already have a Slack App, click the following button to create one:

[`Create your Slack app`](https://api.slack.com/apps/new)

![ssc1](../image/ssc1.png)

* Click on Create an App button then one popup will come.

![ssc2](../image/ssc2.png)

* Choose `From scratch` option. Again one more popup will come.

![ssc3](../image/ssc3.png)

* Enter the app name and Pick a workspace to develop your app and click `Create App` button.

![ssc4](../image/ssc4.png)

### 2. Install the App

* Click the `Incoming Webhooks` feature in the navigation menu and turn on the Activate Incoming Webhooks.

![ssc5](../image/ssc5.png)

* Click the `Install App` settings in the navigation menu. 

![ssc6](../image/ssc6.png)

* Click `Install to workspace` you will redirect to the another page and give the permission to access your app by selecting the channel name or username.

![ssc7](../image/ssc7.png)

### 3. Creating a Slash Commands

Now let's get to actually creating that command. First, head to your [app's management dashboard](https://api.slack.com/apps), select your app and then click the Slash Commands feature in the navigation menu.

You'll be presented with a button marked `Create New Command`, and when you click on it, you'll see a screen where you'll be asked to define your new Slash Command:


* **Command** - the name of command, the actual string that users will type to trigger a world of magic. Bear in mind the naming advice below when you pick this.

* **Request URL** - the URL we'll send a payload to, when the command is invoked by a user. You'll want to use a URL that you can setup to receive these payloads as we'll describe later in this doc. If public distribution is active for your app, this needs to be an HTTPS URL (and self-signed certificates are not allowed). If you're building an app solely for your own workspace, it should also be HTTPS.

* **Short Description** - exactly what it sounds like, a short description of what your command does.

* **Usage Hint** - displayed to users when they try to invoke the command, so if you have any parameters that can be used with your command, we recommend showing them here. You'll see a preview of the autocomplete entry where this hint is displayed, so make sure you're keeping this hint brief enough not to get truncated.

* **Escape channels, users, and links sent to your app** - turning this on will modify the parameters sent with a command by a user. It will wrap URLs in angle brackets (ie. `http://example.com`) and it will translate channel or user mentions into their correlated IDs - so if a user invoked your command like this:

        /addtodo Blockchain

You'll receive the following in the sent data payload:

    Blockchain

If disabled, the payload will just repeat the plain text:

    Blockchain
    
    
We recommend that you enable this feature if you expect to receive user or channel mentions in the command text.


`Escaped usernames may include the plain text username as well, with a bar (|) separator (like this <@U012ABCDEF|worf>).
However, as we are phasing out usernames, your app should only account for the possible presence of that plain text username and separator, but not rely on them.`

**A slight digression about naming your Slash Command**

Consider your command's name carefully. Slash Commands are not namespaced.

This means multiple commands may occupy the same name. If it happens and a user tries to invoke the command, Slack will always invoke the one that was installed most recently. It's an important thing to consider especially if you're planning to distribute your app.

So when you're picking a command name, you will want to avoid terms that are generic and therefore likely to be duplicated. On the other hand, you don't want the command to be too complicated for users to easily remember.

In essence, a great command is descriptive and simple but also unique. Naming it after your service is often a good idea.

Once you've created a truly unforgettable command, any channel or workspace where your app is installed will immediately be able to start using it, so let's learn what to do when a user types one of your app's commands.

### 4. Preparing your app to receive Commands

When a slash command is invoked, Slack sends an HTTP POST to the Request URL you specified above. This request contains a data payload describing the source command and who invoked it, like a really detailed knock at the door.

For example, imagine a workspace at example.slack.com installed an app with a command called /weather. If someone on that workspace types /weather 94070 in their #test channel and submits it, the following payload would be sent to the app:

    token=gIkuvaNzQIHg97ATvDxqgjtO
    &team_id=T0001
    &team_domain=example
    &enterprise_id=E0001
    &enterprise_name=Globular%20Construct%20Inc
    &channel_id=C2147483705
    &channel_name=test
    &user_id=U2147483697
    &user_name=Steve
    &command=/weather
    &text=94070
    &response_url=https://hooks.slack.com/commands/1234/5678
    &trigger_id=13345224609.738474920.8088930838d88f008e0
    &api_app_id=A123456

This data will be sent with a `Content-type` header set as `application/x-www-form-urlencoded`. Here are details of some, but not all, of the important fields you might see in this payload:



| Parameter | Description   |
| --- | --- |
| `token` | This is a verification token, a deprecated feature that you shouldn't use any more. It was used to verify that requests were legitimately being sent by Slack to your app, but you should use the signed secrets functionality to do this instead. |
| `command` | The command that was typed in to trigger this request. This value can be useful if you want to use a single Request URL to service multiple Slash Commands, as it lets you tell them apart. |
| `text` | This is the part of the Slash Command after the command itself, and it can contain absolutely anything that the user might decide to type. It is common to use this text parameter to provide extra context for the command.   |
| `response_url` | A temporary webhook URL that you can use to generate messages responses.   |
| `trigger_id` | A short-lived ID that will let your app open a modal.   |
| `user_id` | The ID of the user who triggered the command.   |
| `user_name` | The plain text name of the user who triggered the command. Do not rely on this field as it is being phased out, use the `user_id` instead.   |
| `team_id`, `enterprise_id`, `channel_id`, etc. | These IDs provide context about where the user was in Slack when they triggered your app's command (eg. which workspace, Enterprise Grid, or channel). You may need these IDs for your command response. |
| `api_app_id` | Your Slack app's unique identifier. Use this in conjunction with request signing to verify context for inbound requests.   |


If public distribution is active for your app, Slack will occasionally send your command's request URL a simple POST request to verify the server's SSL certificate. These requests will include a parameter ssl_check set to 1 and a token parameter.

The token value corresponds to the verification token registered with your app's slash command. See the token field above for more information on validating verification tokens. Mostly, you may ignore these requests, but please do confirm receipt as below.

This payload is like getting all the ingredients to bake a really nice cake, so let's take a look at the recipe.

### 5. Responding to Commands 

There are three main ingredients in the response cake:

* Acknowledge your receipt of the payload.
* Do something useful in response right away.
* Do something useful in response later.

The first is like the cake itself, a required minimum, but the other two are like optional icing and toppings. We'll examine this more closely and get our fingers sticky.

#### Confirming receipt

This is the step which lets Slack, and therefore the user, know that the command was successfully received by the app, regardless of what the app intends to do.

Your app can do this simply by sending back an empty HTTP 200 response to the original request.

If you don't do this, the user will be shown an error message that indicates that the slash command didn't work - not a great experience for the user, so you should always acknowledge receipt (unless of course you didn't receive the command, but then you wouldn't know not to respond, and now we've fallen into a logical paradox).

This confirmation must be received by Slack within 3000 milliseconds of the original request being sent, otherwise a `Timeout was reached` will be displayed to the user. If you couldn't verify the request payload, your app should return an error instead and ignore the request.

The HTTP 200 response doesn't have to be empty however, it can contain other useful stuff - a plain cake isn't all that tasty, so maybe we should add some icing.

#### Sending an immediate response

As mentioned, you can include more substantive info in the body of your HTTP 200 response. In fact, you can use any of the complex [formatting](https://api.slack.com/reference/surfaces/formatting) or [Block Kit layout options](https://api.slack.com/messaging/composing/layouts) that are available when sending any message.

You can include this message either as plain text in the response body:

    It's 80 degrees right now.

Or as a JSON payload in the response body, with a `Content-type` header of `application/json`:

    {
        "blocks": [
            {
                "type": "section",
                "text": {
                    "type": "mrkdwn",
                    "text": "*It's 80 degrees right now.*"
                }
            },
            {
                "type": "section",
                "text": {
                    "type": "mrkdwn",
                    "text": "Partly cloudy today and tomorrow"
                }
            }
        ]
    }
    
**Message Visibility**

There's one special feature to response messages - when responding with a JSON payload you can directly control whether the message will be visible only to the user who triggered the command (we call these ephemeral messages), or visible to all members of the channel where the command was triggered.

The `response_type` parameter in the JSON payload controls this visibility, by default it is set to `ephemeral`, but you can specify a value of `in_channel` to post the response into the channel, like this:

    {
        "response_type": "in_channel",
        "text": "It's 80 degrees right now."
    }

When the `response_type` is `in_channel`, both the response message and the initial Slash Command typed by the user will be shared in the channel:

![ssc8](../image/ssc8.png)